In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random
import pandas as pd
import numpy as np
from copy import deepcopy
import os

# electronics = pd.read_csv('amazonRatings.csv', names=['index','userId','productId','Rating','timestamp'])
# electronics = electronics.drop([0])
# electronics = electronics.drop(['index'], axis=1)
# electronics.to_csv('cleanedElec.csv',index=False,header=False)

random.seed(0)

class DataProcess(object):
    def __init__(self, filename):
        self._filename = filename
        self._loadData()
        self._preProcess()
        self._binarize(self._originalRatings)
        # 对'userId'这一列的数据，先去重，然后构成一个用户列表
        self._userPool = set(self._originalRatings['userId'].unique())
        self._itemPool = set(self._originalRatings['itemId'].unique())
        print("user_pool size: ", len(self._userPool))
        print("item_pool size: ", len(self._itemPool))

        self._select_Negatives(self._originalRatings)
        self._split_pool(self._preprocessRatings)

    def _loadData(self):
        self._originalRatings = pd.read_csv(self._filename,names=['uid','mid','rating','timestamp','sust'])
        is_multi_item = self._originalRatings['mid'].value_counts()>2
        self._originalRatings = self._originalRatings[self._originalRatings['mid'].isin(is_multi_item[is_multi_item].index)]
        is_multi_user=self._originalRatings['uid'].value_counts()>6
        self._originalRatings = self._originalRatings[self._originalRatings['uid'].isin(is_multi_user[is_multi_user].index)]
        return self._originalRatings

    def _preProcess(self):
        """
        对user和item都重新编号，这里这么做的原因是因为，模型的输入是one-hot向量，需要把user和item都限制在Embedding的长度之内，
        模型的两个输入的长度分别是user和item的数量，所以要重新从0编号。
        """
        # 1. 新建名为"userId"的列，这列对用户从0开始编号
        user_id = self._originalRatings[['uid']].drop_duplicates().reindex()
        user_id['userId'] = np.arange(len(user_id)) #根据user的长度创建一个数组
        # 将原先的DataFrame与user_id按照"uid"这一列进行合并
        self._originalRatings = pd.merge(self._originalRatings, user_id, on=['uid'], how='left')

        # 2. 对物品进行重新排列
        item_id = self._originalRatings[['mid']].drop_duplicates()
        item_id['itemId'] = np.arange(len(item_id))
        self._originalRatings = pd.merge(self._originalRatings, item_id, on=['mid'], how='left')

        # 按照['userId', 'itemId', 'rating', 'timestamp']的顺序重新排列
        self._originalRatings = self._originalRatings[['userId', 'itemId', 'rating', 'timestamp','sust']]
        print(self._originalRatings)
        print('Range of userId is [{}, {}]'.format(self._originalRatings.userId.min(), self._originalRatings.userId.max()))
        print('Range of itemId is [{}, {}]'.format(self._originalRatings.itemId.min(), self._originalRatings.itemId.max()))

    def _binarize(self, ratings):
        """
        binarize data into 0 or 1 for implicit feedback
        """
        ratings = deepcopy(ratings)
        ratings['rating'][ratings['rating'] > 0] = 1.0
        self._preprocessRatings = ratings
        # print("binary: \n", self._preprocessRatings)

    def _select_Negatives(self, ratings):
        """
        Select al;l negative samples and 100 sampled negative items for each user.
        """
        # 构造user-item表
        interact_status = ratings.groupby('userId')['itemId'].apply(set).reset_index().rename(
            columns={'itemId': 'interacted_items'})
        print("interact_status: \n", interact_status)

        # 把与用户没有产生过交互的样本都当做是负样本
        interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self._itemPool - x)

        # 从上面的全部负样本中随机选99个出来
        interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 99))
        print("after sampling interact_status: \n", interact_status)

        print("select and rearrange columns")
        self._negatives = interact_status[['userId', 'negative_items', 'negative_samples']]

    def _split_pool(self, ratings):
        """leave one out train/test split """
        print("sort by timestamp descend")
        # 先按照'userID'进行分组，然后根据时间戳降序排列
        ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)
        print(ratings)

        # 选取排名第一的数据作为测试集，也就是最新的那个数据
        test = ratings[ratings['rank_latest'] == 1]
        # 选取所有排名靠后的，也就是历史数据当做训练集
        train = ratings[ratings['rank_latest'] > 1]
        # print("test: \n", test)
        # print("train: \n", train)

        print("size of test {0}, size of train {1}".format(len(test), len(train)))

        # 确保训练集和测试集的userId是一样的
        assert train['userId'].nunique() == test['userId'].nunique()

        self.train_ratings = train[['userId', 'itemId', 'rating']]
        self.test_ratings = test[['userId', 'itemId', 'rating']]

    def sample_generator(self, num_negatives):
        # 合并之后的train_ratings的列包括['userId','itemId'，'rating','negative_items']
        train_ratings = pd.merge(self.train_ratings, self._negatives[['userId', 'negative_items']], on='userId')
        # 从用户的全部负样本集合中随机选择num_negatives个样本当做负样本，并产生一个新的名为"negatives"的列
        train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, num_negatives))
        print(train_ratings)

        # 构造模型所需要的数据，分别是输入user、items以及目标分值ratings。
        users, items, ratings = [], [], []
        for row in train_ratings.itertuples():
            # 构造正样本，分别是userId， itemId以及目标分值1
            users.append(int(row.userId))
            items.append(int(row.itemId))
            ratings.append(float(row.rating))
            # 为每个用户构造num_negatives个负样本，分别是userId， itemId以及目标分值0
            for i in range(num_negatives):
                users.append(int(row.userId))
                items.append(int(row.negatives[i]))
                ratings.append(float(0)) # 负样本的ratings为0，直接强行设置为0

        return users, items, ratings
    
    def test_neg_generator(self, num_negatives):
        # 合并之后的test_ratings的列包括['userId','itemId'，'rating','negative_items']
        test_ratings = pd.merge(self.test_ratings, self._negatives[['userId', 'negative_items']], on='userId')
        # 从用户的全部负样本集合中随机选择num_negatives个样本当做负样本，并产生一个新的名为"negatives"的列
        test_ratings['negatives'] = test_ratings['negative_items'].apply(lambda x: random.sample(x, num_negatives))
        print(test_ratings)

        # 构造模型所需要的数据，分别是输入user、items以及目标分值ratings。
        users, items, ratings = [], [], []
        for row in test_ratings.itertuples():
            # 构造正样本，分别是userId， itemId以及目标分值1
            users.append(int(row.userId))
            items.append(int(row.itemId))
            ratings.append(float(row.rating))
            # 为每个用户构造num_negatives个负样本，分别是userId， itemId以及目标分值0
            for i in range(num_negatives):
                users.append(int(row.userId))
                items.append(int(row.negatives[i]))
                ratings.append(float(0)) # 负样本的ratings为0，直接强行设置为0

        return test_ratings

mag = DataProcess('duplicated_sample.csv')

# generating files for test and train ratings
train_ratings = mag.train_ratings
train_ratings.to_csv('ml-1m.train.rating.csv',header=None, index=None)
test_ratings = mag.test_ratings
test_ratings.to_csv('ml-1m.test.rating.csv',header=None, index=None)

# generating file for test negative
ndf = mag.test_neg_generator(99)
ndf['uipair'] = list(zip(ndf.userId, ndf.itemId))

ndf2 = ndf.loc[:, ['uipair','negatives']]
ndf2.to_csv('ml-1m.test.negative.csv',sep='\t', header=None, index=None)


        userId  itemId  rating   timestamp  sust
0            0       0     2.0  1429401600     1
1            1       0     5.0  1425427200     1
2            2       0     5.0  1404604800     1
3            3       0     5.0  1403049600     1
4            4       0     5.0  1400112000     1
...        ...     ...     ...         ...   ...
245454   14812   12162     5.0  1470355200     1
245455   15826   12372     4.0  1515456000     1
245456   18198   12419     5.0  1516147200     1
245457   19770   12931     5.0  1534204800     1
245458    4278   12931     3.0  1534204800     1

[245459 rows x 5 columns]
Range of userId is [0, 21540]
Range of itemId is [0, 19934]
user_pool size:  21541
item_pool size:  19935


C:\Users\zhuho\AppData\Local\Temp\ipykernel_14400\2321854066.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['rating'][ratings['rating'] > 0] = 1.0


interact_status: 
        userId                                   interacted_items
0           0                   {0, 6432, 1986, 421, 1324, 4280}
1           1  {0, 7969, 5505, 5507, 5575, 7679, 8857, 7061, ...
2           2            {0, 64, 12291, 6564, 2279, 12553, 4402}
3           3              {0, 1410, 715, 2667, 3291, 1652, 603}
4           4  {0, 9923, 4455, 10536, 8236, 6701, 6989, 4435,...
...       ...                                                ...
21536   21536   {10947, 10758, 12044, 12237, 8881, 10972, 11709}
21537   21537  {12641, 9890, 9283, 10627, 10056, 12236, 12783...
21538   21538       {18592, 9283, 10054, 9483, 9487, 9495, 9979}
21539   21539     {9314, 9315, 18501, 18502, 18504, 9887, 18495}
21540   21540  {18913, 11811, 10091, 19506, 19507, 19508, 10109}

[21541 rows x 2 columns]


C:\Users\zhuho\AppData\Local\Temp\ipykernel_14400\2321854066.py:83: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 99))


after sampling interact_status: 
        userId                                   interacted_items  \
0           0                   {0, 6432, 1986, 421, 1324, 4280}   
1           1  {0, 7969, 5505, 5507, 5575, 7679, 8857, 7061, ...   
2           2            {0, 64, 12291, 6564, 2279, 12553, 4402}   
3           3              {0, 1410, 715, 2667, 3291, 1652, 603}   
4           4  {0, 9923, 4455, 10536, 8236, 6701, 6989, 4435,...   
...       ...                                                ...   
21536   21536   {10947, 10758, 12044, 12237, 8881, 10972, 11709}   
21537   21537  {12641, 9890, 9283, 10627, 10056, 12236, 12783...   
21538   21538       {18592, 9283, 10054, 9483, 9487, 9495, 9979}   
21539   21539     {9314, 9315, 18501, 18502, 18504, 9887, 18495}   
21540   21540  {18913, 11811, 10091, 19506, 19507, 19508, 10109}   

                                          negative_items  \
0      {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1      {1, 2, 3, 4, 5, 6, 7, 

C:\Users\zhuho\AppData\Local\Temp\ipykernel_14400\2321854066.py:137: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_ratings['negatives'] = test_ratings['negative_items'].apply(lambda x: random.sample(x, num_negatives))


       userId  itemId  rating  \
0           0       0     1.0   
1           1       0     1.0   
2           3       0     1.0   
3           5       1     1.0   
4           7       1     1.0   
...       ...     ...     ...   
21536   15961   19929     1.0   
21537   14871   19930     1.0   
21538   10033   19933     1.0   
21539   11902   12955     1.0   
21540   12616   12959     1.0   

                                          negative_items  \
0      {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1      {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
2      {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
3      {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
4      {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
...                                                  ...   
21536  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
21537  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
21538  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
215

In [11]:
# generating file for sustainable items
susRatings = mag._originalRatings[mag._originalRatings['sust']==1]
susRatings['itemId'].to_csv('sust.csv',header=None, index=None)

In [12]:
print(len(susRatings['itemId'].unique()))

379


### (trial) filter user based on duplicates

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random
import pandas as pd
import numpy as np
from copy import deepcopy
import os
sampled1 = pd.read_csv('sampledData.csv',names=['uid','mid','rating','timestamp','sust'])
sampled1

In [ ]:
noLabel2 = sampled1[sampled1['sust']==0]
is_multi = noLabel2['uid'].value_counts()>5
noLabel3 = noLabel2[noLabel2['uid'].isin(is_multi[is_multi].index)]
noLabel3

In [ ]:
print(len(noLabel3['uid'].unique()))

In [ ]:
withLabel = sampled1[sampled1['sust']==1]

In [ ]:
finalDf = pd.concat([noLabel3,withLabel])
finalDf

In [ ]:
print(len(finalDf['uid'].unique()))